In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, matthews_corrcoef

from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [69]:
dataset = pd.read_csv('final_dataset_3.csv')
dataset = dataset.set_index('Unnamed: 0')


dataset = dataset[dataset.columns.drop(list(dataset.filter(regex='abs')))]
print(dataset.columns)
features = [col for col in dataset.columns if 'perc' in col]
y_labels = [col for col in dataset.columns if 'change_in' in col]
print(features)
#drop_columns = ['eda_time','time', 'gender']
#dataset.drop(drop_columns, axis=1, inplace=True)
#dataset = dataset.rename(columns={"perc_change": "eda_perc_change", "abs_change": "eda_abs_change"})
dataset.sort_values(by='participantID', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)
dataset


Index(['form_time', 'eda_time', 'eda_perc_change', 'perc_bpm', 'perc_ibi',
       'perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_pnn20', 'perc_pnn50',
       'perc_hr_mad', 'perc_sd1', 'perc_sd2', 'perc_s', 'perc_sd1/sd2',
       'perc_breathingrate', 'form_time.1', 'participantID', 'gender',
       'change_in_stressed', 'change_in_amused', 'change_in_nervous',
       'change_in_engaged', 'change_in_lost', 'change_in_disappointed',
       'change_in_excited', 'change_in_frustrated', 'change_in_contempt'],
      dtype='object')
['eda_perc_change', 'perc_bpm', 'perc_ibi', 'perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_pnn20', 'perc_pnn50', 'perc_hr_mad', 'perc_sd1', 'perc_sd2', 'perc_s', 'perc_sd1/sd2', 'perc_breathingrate']


,form_time,eda_time,eda_perc_change,perc_bpm,perc_ibi,perc_sdnn,perc_sdsd,perc_rmssd,perc_pnn20,perc_pnn50,...,gender,change_in_stressed,change_in_amused,change_in_nervous,change_in_engaged,change_in_lost,change_in_disappointed,change_in_excited,change_in_frustrated,change_in_contempt
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,2022-03-02 14:11,2022-03-02 14:20:01,19.174717,1.105372,0.904673,0.758386,0.639516,0.528432,0.652632,0.388471,...,Male,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2022-03-02 14:28,2022-03-02 14:31:56,-36.539961,0.905580,1.104264,1.913173,1.494328,1.810519,1.428571,2.666667,...,Female,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2022-03-02 14:41,2022-03-02 14:45:20,-18.175823,1.233378,0.810781,1.243654,1.474264,1.457498,0.921053,1.246964,...,Male,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,2022-03-02 14:56,2022-03-02 15:03:31,-11.869750,1.259298,0.794093,2.980787,2.251378,2.582955,1.000000,1.470588,...,Male,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2022-03-02 15:18,2022-03-02 15:20:38,12.059950,0.946140,1.056926,1.415998,1.209645,1.223343,1.240385,1.791667,...,Male,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,2022-03-08 15:18,2022-03-08 15:22:44,73.181025,1.276181,0.783588,1.665727,2.400520,3.334103,1.944444,8.750000,...,Male,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2022-03-08 15:48,2022-03-08 15:52:03,-33.060336,1.036094,0.965163,3.190159,2.624728,3.570906,1.191067,2.115385,...,Female,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2022-03-08 16:00,2022-03-08 16:04:14,-12.953205,0.909589,1.099398,0.769442,1.553925,1.968773,1.500000,1.250000,...,Female,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
8,2022-03-08 16:11,2022-03-08 16:13:46,57.804164,1.233832,0.810483,1.226543,0.588983,0.838645,1.757396,1.780220,...,Male,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [70]:
final_dataset = dataset
final_dataset[y_labels][final_dataset[y_labels] == -1] = 0
final_dataset['eda_perc_change'] = final_dataset['eda_perc_change']/100 +1

C:\Users\Rob\AppData\Local\Temp\ipykernel_21608\4293090475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset[y_labels][final_dataset[y_labels] == -1] = 0
C:\Users\Rob\AppData\Local\Temp\ipykernel_21608\4293090475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataset[y_labels][final_dataset[y_labels] == -1] = 0


In [74]:
final_dataset = final_dataset.replace(to_replace=-1.0,value=0)
final_dataset

,form_time,eda_time,eda_perc_change,perc_bpm,perc_ibi,perc_sdnn,perc_sdsd,perc_rmssd,perc_pnn20,perc_pnn50,...,gender,change_in_stressed,change_in_amused,change_in_nervous,change_in_engaged,change_in_lost,change_in_disappointed,change_in_excited,change_in_frustrated,change_in_contempt
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,2022-03-02 14:11,2022-03-02 14:20:01,1.191747,1.105372,0.904673,0.758386,0.639516,0.528432,0.652632,0.388471,...,Male,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2022-03-02 14:28,2022-03-02 14:31:56,0.634600,0.905580,1.104264,1.913173,1.494328,1.810519,1.428571,2.666667,...,Female,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2022-03-02 14:41,2022-03-02 14:45:20,0.818242,1.233378,0.810781,1.243654,1.474264,1.457498,0.921053,1.246964,...,Male,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,2022-03-02 14:56,2022-03-02 15:03:31,0.881303,1.259298,0.794093,2.980787,2.251378,2.582955,1.000000,1.470588,...,Male,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2022-03-02 15:18,2022-03-02 15:20:38,1.120599,0.946140,1.056926,1.415998,1.209645,1.223343,1.240385,1.791667,...,Male,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,2022-03-08 15:18,2022-03-08 15:22:44,1.731810,1.276181,0.783588,1.665727,2.400520,3.334103,1.944444,8.750000,...,Male,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2022-03-08 15:48,2022-03-08 15:52:03,0.669397,1.036094,0.965163,3.190159,2.624728,3.570906,1.191067,2.115385,...,Female,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2022-03-08 16:00,2022-03-08 16:04:14,0.870468,0.909589,1.099398,0.769442,1.553925,1.968773,1.500000,1.250000,...,Female,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
8,2022-03-08 16:11,2022-03-08 16:13:46,1.578042,1.233832,0.810483,1.226543,0.588983,0.838645,1.757396,1.780220,...,Male,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [129]:
def train_loop(model,x_feature,y_labels):
    
    if model == "svm":
        clf = SVC(kernel='rbf')
    elif model == "dt":
        clf = DecisionTreeClassifier(random_state=0)
    elif model == "rf":
        clf = RandomForestClassifier(n_estimators = 50,max_depth=4, random_state=0,min_samples_leaf=1)
    elif model == 'knn':
        clf = KNeighborsClassifier(n_neighbors = 3)
        
        
    
    for j in range(len(y_labels)):
        #print(y_labels[j])

        participantIDs = np.arange(11)
        total_correct = 0
        train_total_points = 0
        train_total_correct = 0
        total_points = 0
        total_correct_pred_0 = 0
        total_correct_pred_1 = 0
        for i, a in enumerate(participantIDs):
            #print("participantID",i)
            train_ids = participantIDs[participantIDs!=i]
            test_ids = participantIDs[participantIDs==i]
            total_train_data = final_dataset.loc[final_dataset['participantID'].isin(train_ids)]
            test_data = final_dataset.loc[final_dataset['participantID'].isin(test_ids)]

            x_train = total_train_data[x_feature]

            x_test = test_data[x_feature] 

            y_train = total_train_data[y_labels[j]]
            y_test = test_data[y_labels[j]]


            clf.fit(x_train, y_train)
            predict = clf.predict(x_test)
            accuracy = predict == y_test
            total_correct += sum(accuracy)

            if i ==0:
                train_preds = clf.predict(x_train)
                predictions = predict
                ground_truth = y_test.values
                train_truth = y_train.values
            else:
                train_preds = np.hstack((train_preds,clf.predict(x_train)))
                predictions = np.hstack((predictions,predict))
                ground_truth = np.hstack((ground_truth,y_test.values))
                train_truth = np.hstack((train_truth,y_train.values))
            #print("total_correct", total_correct)
            total_points += len(y_test)
            train_total_points += len(y_train)
            #print("prediction:",predict)
            #print("true:", y_test.values )


            train_accuracy = clf.predict(x_train) == y_train

            train_total_correct += sum(train_accuracy)
            #print("total_correct", total_correct)


            total_correct_pred_0 += sum(y_test==0)
            total_correct_pred_1 += sum(y_test==1)
            pred_0 = 0
            #print("total_points", total_points)
            #print(sum(accuracy))
            #print("The train accuracy:", clf.score(x_train, y_train))
            #print("The test accuracy:", clf.score(x_test, y_test))

            #print('-'*15)
        """ print(f' predictions were {predictions}')
        print(f' ground truth was {ground_truth}')
        print(f'f1_score was {f1_score(ground_truth,predictions)}')
        print(f'MCC was {matthews_corrcoef(ground_truth,predictions)}')
        print(f' confusion matrix is {confusion_matrix(ground_truth,predictions)}')

        print("train_accuracy for", y_labels[j], ": ", train_total_correct/train_total_points)
        print("test_accuracy for", y_labels[j], ": ", total_correct/total_points)
        print("accuracy for", y_labels[j], " predict 0 : ", total_correct_pred_0/total_points)
        print("accuracy for", y_labels[j], " predict 1 : ", total_correct_pred_1/total_points)
        print()
        print()
        print('-'*50)"""
        train_accuracy = train_total_correct/train_total_points
        test_accuracy = total_correct/total_points

        return f1_score(ground_truth,predictions), f1_score(train_truth,train_preds),test_accuracy,train_accuracy   
    
        
    
    

In [113]:
print(y_labels)
print(x_features)
y_labels = ['change_in_stressed', 'change_in_amused','change_in_engaged','change_in_frustrated']
selected_features  = ['perc_bpm', 'perc_pnn50', 'perc_s', 'perc_sd1/sd2', 'perc_breathingrate']


['change_in_stressed', 'change_in_amused', 'change_in_engaged', 'change_in_frustrated']
['eda_perc_change', 'perc_breathingrate', 'perc_s']


In [131]:
#x_features_not_eda = x_features[1:]




def select_features(x_features_not_eda,y_label,ML_type):
    best_f1 = -1
    for i in x_features_not_eda:
        #print(['eda_perc_change',i])
        for j in x_features_not_eda:
            if i == j:
                continue

            perf_data = train_loop(ML_type,['eda_perc_change',i,j],y_label)
            F_score = perf_data[0]
            if F_score > best_f1:
                best_f1 = F_score
                best_perf_data = perf_data
                best_features = ['eda_perc_change',i,j]
                #print(f'new best score is {best_f1}')
            
            for k in x_features_not_eda:
                if len(set([i,j,k]))<3:
                    continue
                perf_data = train_loop(ML_type,['eda_perc_change',i,j,k],y_label)
                F_score = perf_data[0]
                if F_score > best_f1:
                    best_f1 = F_score
                    best_perf_data = perf_data
                    best_features = ['eda_perc_change',i,j,k]
                    #print(f'new best score is {best_f1}')

                for l in x_features_not_eda:
                    if len(set([i,j,k,l]))<3:
                        continue
                    perf_data = train_loop(ML_type,['eda_perc_change',i,j,k,l],y_label)
                    F_score = perf_data[0]
                    if F_score > best_f1:
                        best_f1 = F_score
                        best_perf_data = perf_data
                        best_features = ['eda_perc_change',i,j,k]
                        #print(f'new best score is {best_f1}')
    return  best_features,  best_perf_data   


In [115]:
count = 0
for i in selected_features:
    print(['eda_perc_change',i])
    for j in selected_features:
        if i == j:
            continue
        count+=1
        
        for k in selected_features:
            if len(set([i,j,k]))<3:
                continue
            count+=1
            for l in selected_features:
                if len(set([i,j,k,l]))<4:
                    continue
                count+=1

print(count)


['eda_perc_change', 'perc_bpm']
['eda_perc_change', 'perc_pnn50']
['eda_perc_change', 'perc_s']
['eda_perc_change', 'perc_sd1/sd2']
['eda_perc_change', 'perc_breathingrate']
200


In [132]:
for model in ['dt','svm','rf']:
    for y in y_labels:
        best_feats, best_perf_data = select_features(selected_features,[y],model)
        print(f'for ML model {model} on {y} best features were {best_feats}, test F score was {best_perf_data[0]}, F_train was {best_perf_data[1]}, test _accuracy was {best_perf_data[2]}, train accuracy was {best_perf_data[3]}')


for ML model dt on change_in_stressed best features were ['eda_perc_change', 'perc_s', 'perc_pnn50', 'perc_bpm'], test F score was 0.9166666666666666, F_train was 1.0, test _accuracy was 0.875, train accuracy was 1.0
for ML model dt on change_in_amused best features were ['eda_perc_change', 'perc_s', 'perc_breathingrate'], test F score was 0.8000000000000002, F_train was 1.0, test _accuracy was 0.75, train accuracy was 1.0
for ML model dt on change_in_engaged best features were ['eda_perc_change', 'perc_bpm', 'perc_s', 'perc_sd1/sd2'], test F score was 0.875, F_train was 1.0, test _accuracy was 0.875, train accuracy was 1.0
for ML model dt on change_in_frustrated best features were ['eda_perc_change', 'perc_bpm', 'perc_pnn50', 'perc_s'], test F score was 0.7692307692307692, F_train was 1.0, test _accuracy was 0.8125, train accuracy was 1.0
for ML model svm on change_in_stressed best features were ['eda_perc_change', 'perc_bpm', 'perc_pnn50'], test F score was 0.8148148148148148, F_trai

In [133]:
for model in ['knn']:
    for y in y_labels:
        best_feats, best_perf_data = select_features(selected_features,[y],model)
        print(f'for ML model {model} on {y} best features were {best_feats}, test F score was {best_perf_data[0]}, F_train was {best_perf_data[1]}, test _accuracy was {best_perf_data[2]}, train accuracy was {best_perf_data[3]}')


for ML model knn on change_in_stressed best features were ['eda_perc_change', 'perc_bpm', 'perc_sd1/sd2', 'perc_breathingrate'], test F score was 0.846153846153846, F_train was 0.846153846153846, test _accuracy was 0.75, train accuracy was 0.75
for ML model knn on change_in_amused best features were ['eda_perc_change', 'perc_bpm', 'perc_pnn50', 'perc_breathingrate'], test F score was 0.8333333333333333, F_train was 0.8658008658008659, test _accuracy was 0.75, train accuracy was 0.80625
for ML model knn on change_in_engaged best features were ['eda_perc_change', 'perc_bpm', 'perc_s', 'perc_breathingrate'], test F score was 0.7999999999999999, F_train was 0.8510638297872339, test _accuracy was 0.8125, train accuracy was 0.86875
for ML model knn on change_in_frustrated best features were ['eda_perc_change', 'perc_bpm', 'perc_pnn50'], test F score was 0.7692307692307692, F_train was 0.9076923076923077, test _accuracy was 0.8125, train accuracy was 0.925


In [89]:
x_features_not_eda = x_features[1:]
print(x_features_not_eda)

['perc_bpm', 'perc_sdnn', 'perc_sdsd', 'perc_rmssd', 'perc_pnn20', 'perc_pnn50', 'perc_sd2', 'perc_s', 'perc_sd1/sd2', 'perc_breathingrate']


In [124]:
#stressed
y_label = ['change_in_stressed', 'change_in_amused','change_in_engaged','change_in_frustrated']
x_features = ['eda_perc_change', 'perc_s', 'perc_pnn50', 'perc_bpm']


outputs = train_loop("dt",x_features,y_label)
print(outputs)

 predictions were [1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.]
 ground truth was [1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1.]
f1_score was 0.9166666666666666
MCC was 0.7125253031944253
 confusion matrix is [[ 3  2]
 [ 0 11]]
train_accuracy for change_in_stressed :  1.0
test_accuracy for change_in_stressed :  0.875
accuracy for change_in_stressed  predict 0 :  0.3125
accuracy for change_in_stressed  predict 1 :  0.6875


--------------------------------------------------
(0.9166666666666666, 1.0, 0.875, 1.0)
